https://en.wikipedia.org/wiki/Cluster_analysis


https://en.wikipedia.org/wiki/Recommender_system

https://datascience.stackexchange.com/questions/118812/best-algorithms-for-clustering-customers-customer-segmentation

https://link.springer.com/article/10.1007/s44196-024-00568-8

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering
Hierarchical

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html#sklearn.cluster.HDBSCAN Heirarchical dbscan